<a href="https://colab.research.google.com/github/TMPenha/Delivery_Growth_Experiment_Analytics/blob/main/01_etl_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0.GIT

In [1]:
%cd /content
!rm -rf Delivery_Growth_Experiment_Analytics

/content


In [2]:
!git clone https://github.com/TMPenha/Delivery_Growth_Experiment_Analytics.git
%cd Delivery_Growth_Experiment_Analytics


Cloning into 'Delivery_Growth_Experiment_Analytics'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
Receiving objects: 100% (12/12), 9.37 KiB | 1.56 MiB/s, done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Resolving deltas: 100% (2/2), done.
/content/Delivery_Growth_Experiment_Analytics


In [3]:
!pwd

/content/Delivery_Growth_Experiment_Analytics


In [4]:
!mkdir -p notebooks
!mkdir -p data
!mkdir -p reports
!mkdir -p src

In [5]:
!ls

data  LICENSE  notebooks  README.md  reports  src


In [6]:
!ls /content

Delivery_Growth_Experiment_Analytics  sample_data


In [7]:
!mv /content/01_etl_spark.ipynb notebooks/

mv: cannot stat '/content/01_etl_spark.ipynb': No such file or directory


In [8]:
!ls notebooks

In [9]:
!git config --global user.name "TMPenha"
!git config --global user.email "thiagomalbar@email.com"

In [10]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [11]:
!git add .

In [12]:
!git commit -m "feat: add ETL spark notebook"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [13]:
import getpass

TOKEN = getpass.getpass("Digite seu token: ")

Digite seu token: ··········


In [14]:
!git push https://{TOKEN}@github.com/TMPenha/Delivery_Growth_Experiment_Analytics.git


Everything up-to-date


#01. ETL

 ETL com Apache Spark - Growth Experiment Analytics



In [21]:
# 1️⃣ Instala (Colab-friendly)
!pip install -q pyspark
!pip install ijson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 6.9 MB/s eta 0:00:00


In [22]:
# 2️⃣ Importa Spark

import gzip, shutil, tarfile, urllib.request , json, os , ijson
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, to_date, when, lit



In [23]:
# Cria SparkSession
spark = SparkSession.builder \
    .appName("Delivery Growth Experiment ETL") \
    .getOrCreate()

print("✅ Spark Session criada com sucesso!")

✅ Spark Session criada com sucesso!


In [24]:
spark = SparkSession.builder \
    .appName("Delivery Growth Experiment ETL") \
    .getOrCreate()

print("Spark Session criada com sucesso!")

Spark Session criada com sucesso!


In [25]:
# 1️⃣ Instalações e configurações iniciais
!apt-get install openjdk-11-jdk -qq
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark pyspark
!pip install ijson


E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/o/openjdk-lts/openjdk-11-jre-headless_11.0.29%2b7-1ubuntu1%7e22.04_amd64.deb  404  Not Found [IP: 91.189.91.81 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/o/openjdk-lts/openjdk-11-jre_11.0.29%2b7-1ubuntu1%7e22.04_amd64.deb  404  Not Found [IP: 91.189.91.81 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/o/openjdk-lts/openjdk-11-jdk-headless_11.0.29%2b7-1ubuntu1%7e22.04_amd64.deb  404  Not Found [IP: 91.189.91.81 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/o/openjdk-lts/openjdk-11-jdk_11.0.29%2b7-1ubuntu1%7e22.04_amd64.deb  404  Not Found [IP: 91.189.91.81 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?
tar: spark-3.4.1-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [26]:
BASE_DIR = "/content/data"

RAW_DIR = f"{BASE_DIR}/raw"
BRONZE_DIR = f"{BASE_DIR}/bronze"

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(BRONZE_DIR, exist_ok=True)

print("📁 Estrutura de pastas criada")

📁 Estrutura de pastas criada


03. Download e extração dos dados (RAW)

Orders (JSON.GZ – arquivo grande)

In [ ]:
orders_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"
orders_path = f"{RAW_DIR}/order.json.gz"

if not os.path.exists(orders_path):
    urllib.request.urlretrieve(orders_url, orders_path)
    print("✅ orders baixado")
else:
    print("ℹ️ orders já existe")

Consumers

In [ ]:
consumers_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
consumers_gz = f"{RAW_DIR}/consumer.csv.gz"
consumers_csv = f"{RAW_DIR}/consumer.csv"

urllib.request.urlretrieve(consumers_url, consumers_gz)

with gzip.open(consumers_gz, "rb") as f_in, open(consumers_csv, "wb") as f_out:
    shutil.copyfileobj(f_in, f_out)

print("✅ consumers extraído")

Restaurants

In [ ]:

restaurants_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
restaurants_gz = f"{RAW_DIR}/restaurant.csv.gz"
restaurants_csv = f"{RAW_DIR}/restaurant.csv"

urllib.request.urlretrieve(restaurants_url, restaurants_gz)

with gzip.open(restaurants_gz, "rb") as f_in, open(restaurants_csv, "wb") as f_out:
    shutil.copyfileobj(f_in, f_out)

print("✅ restaurants extraído")

AB Test Reference

In [ ]:
ab_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
ab_tar = f"{RAW_DIR}/ab_test_ref.tar.gz"

urllib.request.urlretrieve(ab_url, ab_tar)

with tarfile.open(ab_tar, "r:gz") as tar:
    tar.extractall(path=RAW_DIR)

print("✅ ab_test_ref extraído")

04. Leitura dos dados no Spark (RAW)

Orders (solução correta para JSON gigante)

In [ ]:
orders_df = (
    spark.read
    .option("mode", "PERMISSIVE")
    .option("columnNameOfCorruptRecord", "_corrupt_record")
    .json(orders_path)
)

Consumers / Restaurants / AB Test

In [ ]:
consumers_df = spark.read.option("header", True).csv(consumers_csv)
restaurants_df = spark.read.option("header", True).csv(restaurants_csv)
ab_test_df = spark.read.option("header", True).csv(f"{RAW_DIR}/ab_test_ref.csv")

05. Validação da camada RAW (output agrupado)

In [ ]:
print("📊 DATASETS – CAMADA RAW\n")

datasets_summary = []

# Orders
orders_total = orders_df.count()
corrupt_count = (
    orders_df.filter(F.col("_corrupt_record").isNotNull()).count()
    if "_corrupt_record" in orders_df.columns else 0
)
datasets_summary.append(("orders", orders_total, corrupt_count))

# Consumers
datasets_summary.append(("consumers", consumers_df.count(), 0))

# Restaurants
datasets_summary.append(("restaurants", restaurants_df.count(), 0))

# AB Test
datasets_summary.append(("ab_test", ab_test_df.count(), 0))

In [ ]:
summary_raw_df = spark.createDataFrame(
    datasets_summary,
    ["dataset", "total_registros", "registros_corrompidos"]
)

summary_raw_df = summary_raw_df.withColumn(
    "status",
    F.when(F.col("registros_corrompidos") > 0, "⚠️ ATENÇÃO")
     .otherwise("✅ OK")
)

summary_raw_df.show(truncate=False)

06. Ajustes básicos de tipos (RAW → BRONZE)

In [ ]:
from pyspark.sql.functions import col, to_date


In [ ]:
orders_bronze_df = (
    orders_df
    # Coordenadas
    .withColumn("delivery_address_latitude", col("delivery_address_latitude").cast("double"))
    .withColumn("delivery_address_longitude", col("delivery_address_longitude").cast("double"))
    .withColumn("merchant_latitude", col("merchant_latitude").cast("double"))
    .withColumn("merchant_longitude", col("merchant_longitude").cast("double"))

    # Datas
    .withColumn(
        "order_scheduled_date",
        to_date(col("order_scheduled_date"))
    )
)


In [ ]:
consumers_bronze_df = (
    consumers_df
    .withColumn("active", col("active").cast("boolean"))
)


In [ ]:
restaurants_bronze_df = (
    restaurants_df
    .withColumn("created_at", to_timestamp(col("created_at")))
    .withColumn("enabled", col("enabled").cast("boolean"))
    .withColumn("price_range", col("price_range").cast("int"))
    .withColumn("average_ticket", col("average_ticket").cast("double"))
    .withColumn("takeout_time", col("takeout_time").cast("int"))
    .withColumn("delivery_time", col("delivery_time").cast("int"))
    .withColumn("minimum_order_value", col("minimum_order_value").cast("double"))
)


In [ ]:
ab_test_bronze_df = ab_test_df


07. Escrita da camada BRONZE (Parquet)

In [ ]:
spark.catalog.clearCache()

In [ ]:
orders_df = orders_df.cache()
consumers_df = consumers_df.cache()
restaurants_df = restaurants_df.cache()
ab_test_df = ab_test_df.cache()

# força a execução
orders_df.count()
consumers_df.count()
restaurants_df.count()
ab_test_df.count()


In [ ]:
import shutil
import os

if os.path.exists(BRONZE_DIR):
    shutil.rmtree(BRONZE_DIR)


In [ ]:
orders_df.write.mode("overwrite").parquet(f"{BRONZE_DIR}/orders")
consumers_df.write.mode("overwrite").parquet(f"{BRONZE_DIR}/consumers")
restaurants_df.write.mode("overwrite").parquet(f"{BRONZE_DIR}/restaurants")
ab_test_df.write.mode("overwrite").parquet(f"{BRONZE_DIR}/ab_test")

print("✅ Camada BRONZE criada com sucesso")


08. Validação da camada BRONZE

In [ ]:
print("📊 DATASETS – CAMADA BRONZE\n")

bronze_summary = [
    ("orders", spark.read.parquet(f"{BRONZE_DIR}/orders").count()),
    ("consumers", spark.read.parquet(f"{BRONZE_DIR}/consumers").count()),
    ("restaurants", spark.read.parquet(f"{BRONZE_DIR}/restaurants").count()),
    ("ab_test", spark.read.parquet(f"{BRONZE_DIR}/ab_test").count())
]


In [ ]:
spark.createDataFrame(
    bronze_summary,
    ["dataset", "total_registros"]
).show(truncate=False)


09. Encerramento

In [ ]:
print("🎯 ETL concluído com sucesso.")
print("Pronto para o notebook 02_ab_analysis.ipynb")


🧩 DRAW.iO

In [ ]:
orders_df = spark.read.parquet("/content/data/bronze/orders")
consumers_df = spark.read.parquet("/content/data/bronze/consumers")
restaurants_df = spark.read.parquet("/content/data/bronze/restaurants")
ab_test_df = spark.read.parquet("/content/data/bronze/ab_test")


In [ ]:
def print_table_schema(table_name, df, pk=None, fks=None):
    print(f"\n📦 TABLE: {table_name}")
    print("-" * 50)

    for field in df.schema.fields:
        col = field.name
        dtype = field.dataType.simpleString()

        tags = []
        if pk and col in pk:
            tags.append("PK")
        if fks and col in fks:
            tags.append(f"FK → {fks[col]}")

        tag_str = f" [{', '.join(tags)}]" if tags else ""
        print(f"- {col}: {dtype}{tag_str}")


In [ ]:
print("🧩 MODELO RELACIONAL – DATASETS BRONZE")
print("=" * 60)

# Orders
print_table_schema(
    table_name="orders",
    df=orders_df,
    pk=["order_id"],
    fks={
        "customer_id": "consumers.customer_id",
        "merchant_id": "restaurants.id"
    }
)

# Consumers
print_table_schema(
    table_name="consumers",
    df=consumers_df,
    pk=["customer_id"]
)

# Restaurants
print_table_schema(
    table_name="restaurants",
    df=restaurants_df,
    pk=["id"]
)

# AB Test
print_table_schema(
    table_name="ab_test",
    df=ab_test_df,
    pk=None,
    fks={
        "customer_id": "consumers.customer_id"
    }
)


In [ ]:
def preview_table(df, table_name, n=5):
    print(f"\n📦 TABELA: {table_name}")
    print("-" * 60)
    print("Schema:")
    df.printSchema()
    print("\nAmostra de dados:")
    df.show(n, truncate=False)


In [ ]:
preview_table(orders_df, "orders")
preview_table(consumers_df, "consumers")
preview_table(restaurants_df, "restaurants")
preview_table(ab_test_df, "ab_test")